In [1]:
from pdf2image import convert_from_path           # The convert_from_path function imported from the pdf2image library serves the purpose of converting PDF files into a list of PIL (Python Imaging Library) image objects.

In [18]:
pages = convert_from_path(r"docs\prescription\pre_1.pdf", poppler_path= r"C:\poppler-24.02.0\Library\bin")     # The poppler_path parameter enables specifying the path to the Poppler utility, essential for tools like pdf2image to efficiently interpret PDF contents, simplifying PDF manipulation in your programs.

In [19]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200>]

In [20]:
type(pages)

list

In [21]:
len(pages)

1

In [5]:
# Currently i got only one image as there was only one page in the pdf

In [7]:
# this command will show the image
pages[0].show()

In [6]:
import pytesseract

In [22]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"      # This line of code ensures that pytesseract knows where to find the Tesseract OCR tool{a.k.a.  google tesseract engine} so it can be used for text recognition tasks.
text = pytesseract.image_to_string(pages[0], lang = 'eng')                                          # `image_to_string()` function is used to convert an image into text. can also specify language, if the image has text in diffrent language then can specify that
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Maria Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

—moemenmannenneneneunmnnnnennieesisiyoinnitniahadaaanniihsnseneneneeeernnttnnneenrenen:

Prednisone 20 mg
Lialda 2.4 gram

3 days,

or 1 month


In [12]:
# now we got the text from the image, but note it is not perfect, we were not able to get the text part that was in shadow.
# this means that i cannot apply this tesseract directly to the image because the image is not perfect , i have to apply some image preprocessing before using the tesseract to extract text from the image.

In [24]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)      # `np.array` will convert the image into array form, and our image is a scanned image,it could be in coloured form,so first i have to convert that into greyscale form{black and white form}, which simplifies the image for further processing. For that we use `cv2.COLOR_BGR2GRAY`
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)  # this involves changing the dimension of the image. Here, image enlarged by 1.5 in both horizontal and vertical directions{this 1.5 value is chosen after trial and error}
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

* `fx` stands for "factor along the x-axis" and `fy` stands for "factor along y-axis"
* The `None` value passed as the second argument to `cv2.resize` indicates that the output size of the resized image is not explicitly defined. When `None` is provided, the function automatically calculates the dimensions of the output image based on the scaling factors provided in `fx` and `fy`.
  If we specify output size then no need to specify `fx` and `fy` values.
* `interpolation`: thsi parameter specifies teh method used to interpolate between pixel values when resizing. Here, "cv2.INTER_LINEAR" means linear interpolation is used. Linear interpolation calculates new pixel values based on weighted averages of surrounding pixels.

**Overall, the resizing step increases the size of the grayscale image to enhance features or details, making them easier to detect in further processing steps.**

In [25]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [26]:
# now will apply the teseract to extract text from the post processed image
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(img, lang = 'eng')
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Marta Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

K

Prednisone 20 mg
Lialda 2.4 gram

Directions:

Prednisone, Taper 5 mig every 3 days,
Finish in 2.5 weeks a
Lialda - take 2 pill everyday for 1 month

Refill: 2 times


**Note**: after post-processinh we are able to extract all the texts

Now we apply the same program on the second pdf from the "Docs" folder.


**note** after the processing we got the image a little tilted, due to which when we extrcat the text we will find error

Clearly can see in place of "S" it has extracted ">" etc. This shows that project will **not give 100 percent accurcay**. With this automation, we might have to corrcet some of the errors manually. So, **our project is a combination of automation and human effort**.
But still it mostly working fine.